In [2]:
import scipy.io as sio
import numpy as np
from scipy.signal import butter, filtfilt, welch
from sklearn.decomposition import FastICA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from mne.decoding import CSP
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# Define bandpass filter function
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data, axis=0)

# Define function to process data
def process_participant(data_path, classifier_name):
    # Load the dataset
    data = sio.loadmat(data_path)
    cnt = 0.1 * np.double(data['cnt'])

    # Bandpass filter the data
    fs = data['nfo']['fs'][0][0]
    filtered_data = bandpass_filter(cnt, 8.0, 15.0, fs)

    # Perform ICA
    ica = FastICA(n_components=59, random_state=0)
    ica_data = ica.fit_transform(filtered_data)
    ica_data = ica_data @ ica.mixing_.T

    # Time window segmentation
    marker_positions = data['mrk']['pos'][0][0].flatten()
    start_sample = int(0.5 * fs)
    end_sample = int(2.5 * fs)
    epoch_length = end_sample - start_sample
    n_trials = len(marker_positions)
    trials = np.zeros((n_trials, 59, epoch_length))

    for i in range(n_trials):
        start_pos = int(marker_positions[i])
        start_idx = start_pos + start_sample
        end_idx = start_pos + end_sample
        trials[i] = ica_data[start_idx:end_idx, :].T

    labels = data['mrk']['y'][0][0].flatten()

    # Apply CSP
    csp = CSP(n_components=6, reg=None, log=True, cov_est='epoch')
    csp.fit(trials, labels)
    csp_features = csp.transform(trials)

    # Split the CSP features into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(csp_features, labels, test_size=0.2, random_state=42)

    # Handle classifier initialization
    if classifier_name == 'KNN':
        classifier = KNeighborsClassifier(n_neighbors=100, metric='mahalanobis', metric_params={'VI': np.cov(X_train, rowvar=False)}, weights='distance')
    else:
        classifier = classifiers[classifier_name]

    # Train and evaluate the classifier
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# List of participant data files
participants = ['/Users/junaeidshoaib/Desktop/Dessertation/Dataset/Matlab_49/BCICIV_1_mat/BCICIV_calib_ds1a.mat', 
                '/Users/junaeidshoaib/Desktop/Dessertation/Dataset/Matlab_49/BCICIV_1_mat/BCICIV_calib_ds1b.mat', 
                '/Users/junaeidshoaib/Desktop/Dessertation/Dataset/Matlab_49/BCICIV_1_mat/BCICIV_calib_ds1c.mat', 
                '/Users/junaeidshoaib/Desktop/Dessertation/Dataset/Matlab_49/BCICIV_1_mat/BCICIV_calib_ds1d.mat', 
                '/Users/junaeidshoaib/Desktop/Dessertation/Dataset/Matlab_49/BCICIV_1_mat/BCICIV_calib_ds1e.mat', 
                '/Users/junaeidshoaib/Desktop/Dessertation/Dataset/Matlab_49/BCICIV_1_mat/BCICIV_calib_ds1f.mat', 
                '/Users/junaeidshoaib/Desktop/Dessertation/Dataset/Matlab_49/BCICIV_1_mat/BCICIV_calib_ds1g.mat']

# Define classifiers
classifiers = {
    'SVM': SVC(kernel='rbf', C=0.06, gamma='scale'),
    'LDA': LinearDiscriminantAnalysis(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Complex Tree': DecisionTreeClassifier(max_depth=15, criterion='entropy'),
    'Naive Bayes': GaussianNB()
}

# Process each participant and calculate accuracy
results = {clf_name: [] for clf_name in classifiers}

for clf_name in classifiers:
    print(f"Processing classifier: {clf_name}")
    for participant in participants:
        accuracy = process_participant(participant, clf_name)
        results[clf_name].append(accuracy)
        print(f"Participant: {participant}, Accuracy: {accuracy:.2f}")

# Adding KNN separately because it requires specific initialization with X_train data
knn_accuracies = []
for participant in participants:
    accuracy = process_participant(participant, 'KNN')
    knn_accuracies.append(accuracy)
    print(f"KNN - Participant: {participant}, Accuracy: {accuracy:.2f}")

results['KNN'] = knn_accuracies

# Display mean accuracy for each classifier
mean_accuracies = {clf_name: np.mean(acc) for clf_name, acc in results.items()}
for clf_name, mean_acc in mean_accuracies.items():
    print(f"Mean accuracy for {clf_name}: {mean_acc:.2f}")


Processing classifier: SVM
Computing rank from data with rank=None
    Using tolerance 1.3e+02 (2.2e-16 eps * 59 dim * 1e+16  max singular value)
    Estimated rank (data): 59
    data: rank 59 computed from 59 data channels with 0 projectors
Reducing data rank from 59 -> 59
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance usin

Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estima

/Users/junaeidshoaib/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


    Using tolerance 31 (2.2e-16 eps * 59 dim * 2.4e+15  max singular value)
    Estimated rank (data): 59
    data: rank 59 computed from 59 data channels with 0 projectors
Reducing data rank from 59 -> 59
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.

Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estima

...

Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMP

Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMP

Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estima

Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estima

Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=-1 covariance using EMP

In [4]:
# Calculate mean accuracy for each classifier
mean_accuracies = {clf_name: np.mean(accs) for clf_name, accs in results.items()}

print("Results per classifier:")
for clf_name, accs in results.items():
    print(f"{clf_name} - Mean Accuracy: {mean_accuracies[clf_name]:.2f}")
    for i, acc in enumerate(accs):
        print(f"  Participant {i + 1}: {acc:.2f}")

Results per classifier:
SVM - Mean Accuracy: 0.93
  Participant 1: 0.90
  Participant 2: 0.93
  Participant 3: 0.97
  Participant 4: 0.95
  Participant 5: 0.97
  Participant 6: 0.90
  Participant 7: 0.90
LDA - Mean Accuracy: 0.93
  Participant 1: 0.90
  Participant 2: 0.93
  Participant 3: 0.97
  Participant 4: 0.95
  Participant 5: 0.95
  Participant 6: 0.85
  Participant 7: 0.95
Logistic Regression - Mean Accuracy: 0.94
  Participant 1: 0.88
  Participant 2: 0.90
  Participant 3: 0.97
  Participant 4: 0.95
  Participant 5: 0.97
  Participant 6: 0.93
  Participant 7: 0.97
Complex Tree - Mean Accuracy: 0.89
  Participant 1: 0.90
  Participant 2: 0.88
  Participant 3: 0.93
  Participant 4: 0.93
  Participant 5: 0.90
  Participant 6: 0.80
  Participant 7: 0.88
Naive Bayes - Mean Accuracy: 0.93
  Participant 1: 0.90
  Participant 2: 0.93
  Participant 3: 0.97
  Participant 4: 0.95
  Participant 5: 0.95
  Participant 6: 0.88
  Participant 7: 0.93
KNN - Mean Accuracy: 0.92
  Participant 1: 